In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
import helper_funcs
import torch
import numpy as np
import faiss
import random

/home/mputo/.conda/envs/search/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
encoder = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
#cross_encoder = CrossEncoder('cross-encoder/stsb-distilroberta-base')

In [ ]:
d = 384
description = "Flat"
#description = "IVF1024(PQ16x4),PQ16"
# description = "HNSW32_SQ8"
# description = "HNSW32_PQ4"
# description = "PCA64,IVF32(PQ2x4),Flat"
# description = "OPQ16,PCA64,IVF32(PQ2x4),Flat"
# description = "IVF64(PQ4x6),Flat"
# description = "IVF64(SQ8),Flat"
# description = "IVF64(PQ6x6),Flat"

index = helper_funcs.create_index(d, description, gpu=True)

print(index.ntotal)

0


In [ ]:
files = []
for i in range(1, 10):
    f = torch.load(f'../msmarco-vec/msmarco-vectors-{i}.pt')
    files.append(f)

xb = np.concatenate(files)

In [ ]:
print(xb.shape)

(1000000, 384)


In [ ]:
def populate_index(xb, index):
    faiss.normalize_L2(xb)

    index.train(xb)
    index.add(xb)

In [ ]:
populate_index(xb, index)

print(index.ntotal)

1000000


In [ ]:
phase = "dev.small"

collection = helper_funcs.tsv_to_df("collectionandqueries/collection", col_name=["id", "paragraph"], index_col="id")
queries = helper_funcs.tsv_to_df(f"collectionandqueries/queries.{phase}", col_name=["query number", "query text"], index_col="query number")
qrels = helper_funcs.tsv_to_df(f"collectionandqueries/qrels.{phase}", col_name=["query id", "del.", "document id", "del.2"], index_col="query id").drop(labels=["del.", "del.2"], axis=1)

In [ ]:
seeds = ['e5uyHzt3gc', 'bxL9fUAPzw', 'SF1jpDpjd5', '3WoYWkOWhY', '3bHYRrzvIa', 'DpImBXEsfL', '43dsJs4PP6', 'RbNwkeOSMS', 'qY7pzPrsnX', 'A9NJJ1J9cC']

for s in seeds:
    random.seed(s)
    print(random.randint(0, 6980))

2913
627
4094
5153
3949
2712
4845
5361
2605
5249


In [ ]:
keys = [342115, 1069981, 671692, 987237, 927553, 68095, 447551, 992840, 272605, 989894]


for key in keys:
    query, document = helper_funcs.get_query_document_pair(key, collection, queries, qrels)


    query_val = query.iloc[0]["query text"]
    document_val = document.iloc[0]["paragraph"]

    print("Query: ", query_val)
    print("Document: ", document_val)

    reranked_results = helper_funcs.search_and_rerank(query_val, 10, encoder=encoder, cross_encoder=cross_encoder, index=index, collection=collection)

    answer = helper_funcs.generate_answer(query_val, reranked_results)

    print(answer)
    print('-------HITS--------')
    print(reranked_results)
    print('-------------------')

IndexError: index 2913 is out of bounds for axis 0 with size 1